# **NLP analysis on top artsts' lyrics**

What to we want with NLP analysis?

> Easy thing is to find common words and themes (REMEMBER we are given the genre attribute already).\
> Can NLP account for the mood and sentiment? Even if it is a love song, it could be very sad.\
> Do artists in the same genre also use similar words?\
> Do artists who use similar words tend to collaborate more often? (or artist with the same sentiment and mood)\
> Can genre be guessed from lyrics and sentiment?\
> We also see how well do artists actually represent their genres IF genres can be guessed from lyrics alone? - like sometimes it genre is more a "sound, music" style rather than lyrical style.

Definitions:
> Artist sentiment is the average of the top 5 songs.

Things we should be careful of:
> Since we take artists above a popularity threshold - we should address that this analysis may be limited or closed off.\
> REMEMBER: we already have genres as attributes but the NLP does not know it ofc.